In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datashader as ds
from datashader import transfer_functions as tf
from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno
from datashader.bokeh_ext import create_ramp_legend, create_categorical_legend
import warnings
warnings.filterwarnings('ignore')

from bokeh.io import output_notebook, show

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("csv"))

# Any results you write to the current directory are saved as output.

* **Preliminary note**:
This is a work in progress, will add analysis of air dose rates with proper map legend and more ...

## I. Data loading

In [5]:
# Taking a quick look at the first 10 rows
df = pd.read_csv('csv/measurements.csv', nrows=10)
df.head()

,Captured Time,Latitude,Longitude,Value,Unit,Location Name,Device ID,MD5Sum,Height,Surface,Radiation,Uploaded Time,Loader ID
0,2020-02-03 17:00:00,37.507552,139.941170,72,cpm,NaN,NaN,6449bbf7ce3b30a8e05bc23a0bc40644,NaN,NaN,NaN,2020-02-03 17:00:00,633
1,2020-02-03 11:00:00,37.505445,0.016667,68,cpm,NaN,NaN,a166df14f60b61095693684fc0f89c54,NaN,NaN,NaN,2020-02-03 11:00:00,614
2,2020-02-03 11:00:00,37.507250,139.940000,55,cpm,NaN,NaN,2fa8bccef282796bcdc297679c4db5b3,NaN,NaN,NaN,2020-02-03 11:00:00,614
3,2020-02-01 03:00:00,34.066487,-118.895217,50,cpm,NaN,NaN,da79c21520d3ff3f5ed010a70f4a6d29,NaN,NaN,NaN,2020-02-01 03:00:00,507
4,2020-02-01 01:00:00,37.673233,140.066667,48,cpm,NaN,NaN,6cdf740a2304a850185f74c0c4a4b878,NaN,NaN,NaN,2020-02-01 01:00:00,504


In [ ]:
# Loading all rows but only Latitude, Longitude, Value, Unit and LoaderID
df = pd.read_csv('csv/measurements.csv', usecols=[1, 2, 3, 4, 12], nrows=10)
df

In [7]:
# Number of measurements
print('Number of measurements: ', df.shape[0])

Number of measurements:  10


## II. A bit of data cleaning

In [8]:
# Renaming columns
df.columns = ['lat', 'lon', 'value','unit', 'loader_id']
df.head

<bound method NDFrame.head of          lat         lon  value unit  loader_id
0  37.507552  139.941170     72  cpm        633
1  37.505445    0.016667     68  cpm        614
2  37.507250  139.940000     55  cpm        614
3  34.066487 -118.895217     50  cpm        507
4  37.673233  140.066667     48  cpm        504
5  37.674782  140.079895     52  cpm        504
6  37.517902  139.925478     59  cpm        461
7  37.737123  140.726853    454  cpm        461
8  37.539610  140.116577     47  cpm        461
9  37.755880  140.703412    463  cpm        461>

In [9]:
# Keeping only cpm (counts per minutes)
df = df[df.unit == 'cpm']

In [10]:
# Convert cpm to µSv/h
# http://safecast.org/tilemap/methodology.html
df.value = df.value / 350

In [11]:
# Drop any NA
df.dropna(axis=0, how='any', inplace=True)

In [12]:
# Keep only positive values
df = df[df.value > 0]

In [13]:
# "Casting" Safecast data
df.loader_id = df.loader_id.astype(int)

### III. EDA

### III. 1 Data loaders activity

In [14]:
loaders = df['loader_id'].value_counts()
print("Min. # of measurements loaded: ", loaders.min())
print("Max. # of measurements loaded: ", loaders.max())

Min. # of measurements loaded:  1
Max. # of measurements loaded:  4


In [ ]:
fig, ax = plt.subplots()
loaders.hist(ax=ax, bins=100, bottom=0.1)
ax.set_yscale('log')
ax.set_xlabel("# of measurements")
ax.set_ylabel("# of loaders (Log scale)")

### III. 2 Spatializing number of measurements

In [ ]:
plot_width  = int(800)
plot_height = int(plot_width//1.2)

In [ ]:
def draw_map(df, plot_width, plot_height, colors, agg_func, interp, background_col):
    cvs = ds.Canvas(plot_width=plot_width, plot_height=plot_height)
    agg = cvs.points(df, 'lon', 'lat',  agg_func('value'))
    img = tf.shade(agg, cmap=colors, how=interp)
    return tf.set_background(img, color=background_col)

* **Worldwide**

In [ ]:
# NEED PROPER MAP LEGEND
img = draw_map(df, plot_width, plot_height, inferno, ds.count, 'log', 'black')
img

* **Zoom to Japan**

In [ ]:
x_min_jpn, y_min_jpn, x_max_jpn, y_max_jpn = 128.03, 30.22, 148.65, 45.83
df_jpn = df[(df.lon > x_min_jpn) & (df.lon < x_max_jpn) & (df.lat > y_min_jpn) & (df.lat < y_max_jpn)]

In [ ]:
img = draw_map(df_jpn, plot_width, plot_height, inferno, ds.count, 'log', 'black')
img

* **Around Fukushima, Japan**

In [ ]:
x_min_fk, y_min_fk, x_max_fk, y_max_fk = 140.0166, 37.0047, 141.2251, 38.195
df_fk= df[(df.lon > x_min_fk) & (df.lon < x_max_fk) & (df.lat > y_min_fk) & (df.lat < y_max_fk)]

In [ ]:
img = draw_map(df_fk, plot_width, plot_height, inferno, ds.count, 'log', 'black')
img